Intermodal nx_graph can be loaded from OSM [example](graph_generator.ipynb)

In [ ]:
from objectnat import get_adjacency_matrix, get_intermodal_graph_from_osm, GraphType
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', 15)

nx_graph = get_intermodal_graph_from_osm(city_osm_id=1114252, city_crs=32636,keep_city_boundary=True)
buildings = gpd.read_parquet('examples_data/spb_buildings.parquet').to_crs(32636)
services = gpd.read_parquet('examples_data/spb_services.parquet').to_crs(32636)
services=services[services['city_service_type']=='Фастфуд']
matrix: pd.DataFrame = get_adjacency_matrix(buildings_from=buildings,
                                            services_to=services,
                                            nx_graph=nx_graph,
                                            weight="time_min", 
                                            city_crs=32636,
                                            graph_type=[GraphType.WALK,GraphType.PUBLIC_TRANSPORT]) 

In [ ]:
matrix.to_parquet('examples_data/matrix_time.parquet')